# Multi-Agent Research Assistant
## (using multi-agent collaboration system)

Your task is to develop a multi-agent AI system that can autonomously research a complex
topic, synthesize information from multiple sources, and generate a comprehensive report.
This system should demonstrate agentic behaviors, including planning, reasoning, task
decomposition, and collaboration between specialized agents.
Requirements:
The system must include at least three specialized agents with distinct roles (e.g.,
researcher, analyzer, writer)
Agents must communicate with each other to coordinate their work
The system should be able to handle a research query on any business or technical
topic
It must autonomously plan its approach to the research task
It should search for and retrieve information from the web and/or provided documents
It must evaluate source credibility and resolve conflicting information
The final output should be a well-structured report with proper citations
The system should maintain a traceable chain of reasoning for its conclusions

In this project, we will use 3 agents

- Researcher Agent 
- Analyzer Agent
- Writer Agent


In this project, we will use a **Hierarchial Process**, where crewAI creates a manager to delegate task to all the 3 agents

## Step 01. Installing crewai and crewai_tools libraries

**Note:** crewai requires Python version >=3.10 and <=3.13

In [1]:
import sys
print(sys.version)

3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]


In [2]:
!pip install crewai

  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached auth0_python-4.9.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached chromadb-1.0.9-cp39-abi3-win_amd64.whl.metadata (7.0 kB)
  Using cached click-8.2.0-py3-none-any.whl.metadata (2.5 kB)
  Using cached instructor-1.8.2-py3-none-any.whl.metadata (23 kB)
  Using cached json_repair-0.44.1-py3-none-any.whl.metadata (12 kB)
  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached openai-1.79.0-py3-none-any.whl.metadata (25 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_api-1.33.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.33.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.33.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached pdfplumber-0.11.6-py3-none-any.whl.metadata (42 kB)
  Using cached pydantic-2.11.4-py3-none-any.whl.metadata (66 kB)
  Using cached python


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
pip install crewai_tools

  Using cached crewai_tools-0.45.0-py3-none-any.whl.metadata (10 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached embedchain-0.1.128-py3-none-any.whl.metadata (9.2 kB)
  Using cached lancedb-0.22.0-cp39-abi3-win_amd64.whl.metadata (4.4 kB)
  Using cached pyright-1.1.400-py3-none-any.whl.metadata (6.6 kB)
  Using cached pytube-15.0.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached alembic-1.15.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached gptcache-0.1.44-py3-none-any.whl.metadata (24 kB)
  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_cohere-0.3.5-py3-none-any.whl.metadata (6.7 kB)
  Using cached langchain_community-0.3.24-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_openai-0.2.14-py3-none-any.whl.metadata (2.7 kB)
  Using cached mem0ai-0.1.99-py3-none-any.whl.metadata (7.5 kB)
  Using cached posthog-3.25.0-py2.py3-none-any

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'hnswlib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Step 02. Importing modules, LLMs, tools

**NOTE:** To create Agent class objects, we need to specify OPEN_AI model name and api key as environment variables



In [4]:
from crewai import Crew, Agent, Task

# import os
# os.environ['OPENAI_MODEL_NAME'] = "gpt-4-turbo"
# os.environ['OPENAI_API_KEY'] = "Your OPEN AI Api Key"
# os.environ['SERPER_API_KEY'] = "Your Server API key"

import os

#from langchain_community.chat_models import AzureChatOpenAI
from langchain_openai import AzureChatOpenAI

OPENAI_API_KEY_EXT="f21b6916332f72245df2484f57aa7aaa589d48868fd0842ebb3b1ed55b4b61077f795dc9a2e951738ae0c6752e5037b5281055d"
OPENAI_API_BASE_EXT="dallas-int-prod.com/coreapi/openai/v1"
# Initialize Azure OpenAI

llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    openai_api_key=OPENAI_API_KEY_EXT,
    openai_api_base=OPENAI_API_BASE_EXT,
    model="mmc-tech-gpt-4o-mini-128k-2024-07-18"
)




In [7]:
# pip install langchain_openai

  Using cached langchain_openai-0.3.17-py3-none-any.whl.metadata (2.3 kB)
  Using cached langchain_core-0.3.60-py3-none-any.whl.metadata (5.8 kB)
  Using cached langsmith-0.3.42-py3-none-any.whl.metadata (15 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached zstandard-0.23.0-cp312-cp312-win_amd64.whl.metadata (3.0 kB)
Using cached langchain_openai-0.3.17-py3-none-any.whl (62 kB)
Using cached langchain_core-0.3.60-py3-none-any.whl (437 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached langsmith-0.3.42-py3-none-any.whl (360 kB)
Using cached packaging-24.2-py3-none-any.whl (65 kB)
Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
Using cached zstandard-0.23.0-cp312-cp312-win_amd64.whl (495 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### crewAI Tools

In [14]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

ModuleNotFoundError: No module named 'crewai_tools'

In [15]:
pip install chroma-hnswlib

  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build chroma-hnswlib
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'hnswlib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


## Step 03. Creating Agents

### Agent 1: Research Agent

- Researcher Agent 
- Analyzer Agent
- Writer Agent

In [16]:
research_agent = Agent(
    role="Researcher",

    goal="Autonomously research complex topics, synthesize information from multiple sources, "
         "and generate comprehensive reports that demonstrate insights and actionable recommendations.",
    backstory="With expertise in multi-agent collaboration, this agent specializes in synthesizing "
              "information from diverse sources. Utilizing advanced planning, reasoning, and task decomposition, "
              "the Researcher Agent coordinates with specialized agents to gather, analyze, and present data, "
              "ensuring thorough and insightful reports that inform decision-making.",
    allow_delegation=True,
    verbose=True,
    # tools=[search_tool, scrape_tool]
)


### Agent 2: Analyzer Agent

In [17]:
analyzer_agent = Agent(
    role="Analyzer",
    goal="Analyze and interpret complex data to extract meaningful insights,"
        "identify patterns, and support data-driven decision-making across various domains.",
    backstory="Equipped with a deep understanding of data "
              "research and quantitative analysis, this agent "
              "devises and refines the output. It evaluates "
              "the data of different searches to determine "
              "the most appropriate and risk-averse options.",
    allow_delegation=True,
    # tools=[search_tool, scrape_tool],
    verbose=True
)

### Agent 3: Execution Agent / Writer Agent

In [18]:
execution_agent = Agent(
    role="Writer Agent",
    goal="Craft clear, concise, and engaging reports and documents "
    "that effectively communicate complex information and insights to diverse audiences.",
    backstory="With a talent for storytelling and a deep understanding of various subject matters,"
    "the Writer Agent specializes in translating intricate data and analyses into accessible narratives."
    "By collaborating with research and analysis agents,"
    "this agent ensures that the final outputs are not only informative but also compelling, "
    "making complex topics understandable and actionable for stakeholders.",
    allow_delegation=True,
    # tools=[search_tool, scrape_tool],
    verbose=True
)

## Step 04. Creating Tasks

In [20]:
research_task = Task(
    description=(
        "Conduct in-depth research on the selected topic ({topic_selection}). "
        "Synthesize information from multiple sources to gather insights and "
        "generate a comprehensive report that addresses key questions and findings."
    ),
    expected_output=(
        "A detailed report summarizing the research findings, "
        "including insights, conclusions, and actionable recommendations for {topic_selection}."
    ),
    agent=research_agent,
)


In [21]:
data_analysis_task = Task(
    description=(
        "Analyze the collected data related to {data_source} to identify patterns, "
        "trends, and correlations. Utilize statistical methods and machine learning techniques "
        "to derive meaningful insights that can inform strategic decisions."
    ),
    expected_output=(
        "A set of analytical insights, including identified trends, patterns, "
        "and potential implications for decision-making regarding {data_source}."
    ),
    agent=analyzer_agent,
)



In [22]:
writing_task = Task(
    description=(
        "Draft a clear and engaging report based on the insights gathered from the research "
        "and analysis of {topic_selection}. Ensure that the document is tailored to the target audience "
        "and effectively communicates the key findings and recommendations."
    ),
    expected_output=(
        "A polished report that presents the research and analysis findings in a compelling manner, "
        "including an executive summary, detailed sections, and clear conclusions for {topic_selection}."
    ),
    agent=execution_agent,
)


## Step 06. Creating Crew

### Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [23]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

In [25]:
# define the crew with agent and tasks
research_assistant_crew = Crew (
    agents=[research_agent, 
            analyzer_agent, 
            execution_agent],
    
    tasks=[research_task, 
           data_analysis_task, 
           writing_task],
    
    verbose=True,
    process=Process.hierarchical,
    manager_llm=llm
)

### Running the crew

In [26]:
# # specify the input for crew
# financial_trading_inputs = {
#     'stock_selection': 'AAPL',
#     'initial_capital': '100000',
#     'risk_tolerance': 'Medium',
#     'trading_strategy_preference': 'Day Trading',
#     'news_impact_consideration': True
# }

research_assistant_inputs = {
    'topic_selection': 'Impact of Market Trends on Technology Stocks',
    'data_source': 'Recent Market Data and Financial Reports',
    'audience': 'Investment Analysts and Stakeholders',
    'report_format': 'Executive Summary with Detailed Analysis',
    'research_depth': 'Comprehensive',
    'analysis_techniques': ['Statistical Analysis', 'Machine Learning'],
    'writing_style': 'Professional and Engaging'
}


In [27]:
### this execution will take some time to run
result = research_assistant_crew.kickoff(inputs=research_assistant_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 36542fce-128e-4dcb-a8db-0709a150a3c4                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Provider List: https://docs.litellm.ai/docs/providers



🚀 Crew: crew
└── 📋 Task: 4e51da14-2859-4382-81a5-371d04746ad9
       Status: Executing Task...


Provider List: https://docs.litellm.ai/docs/providers



🚀 Crew: crew
└── 📋 Task: 4e51da14-2859-4382-81a5-371d04746ad9
       Status: Executing Task...
    └── 🤖 Agent: Crew Manager
            Status: In Progress

...

 Error during LLM call: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=mmc-tech-gpt-4o-mini-128k-2024-07-18
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers
 An unknown error occurred. Please check the details below.
 Error details: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=mmc-tech-gpt-4o-mini-128k-2024-07-18
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers


🚀 Crew: crew
└── 📋 Task: 4e51da14-2859-4382-81a5-371d04746ad9
       Assigned to: Crew Manager
       Status: ❌ Failed
    └── 🤖 Agent: Crew Manager
            Status: In Progress
        └── ❌ LLM Failed

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 4e51da14-2859-4382-81a5-371d04746ad9                                                                     │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 36542fce-128e-4dcb-a8db-0709a150a3c4                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=mmc-tech-gpt-4o-mini-128k-2024-07-18
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

- Display the final result as Markdown.

In [36]:
from IPython.display import Markdown
Markdown(result)

The comprehensive risk analysis report and mitigation recommendations for day trading AAPL are as follows:

Risk Analysis:
- **Volume and Liquidity**: Fluctuating trading volumes and liquidity patterns in AAPL indicate potential increased trade costs and price slippage risks.
- **Volatility**: Market shifts influenced by broader economic indicators and policy decisions may lead to sudden price changes in AAPL.
- **Price Segment**: High performance attracting speculative trading in AAPL can result in increased price volatility.
- **Macro Factors**: International trade adjustments and political impacts can indirectly affect market sentiment and AAPL’s stock performance.
- **News and Events**: Any news related to AAPL or its sector can cause sharp movements in stock price.

Mitigation Strategies:
1. **Two-Day High/Low Strategy**: Buying breakouts from a two-day high or selling breakdowns from a two-day low with a 1:2 risk/reward ratio and strict stop-loss measures.
2. **Trading Ranges During Low Volatility**: Utilizing oscillators to identify overbought and oversold levels during low volatility periods.
3. **GAP-FILL Strategy**: Trading the gaps between the previous day's close and the current day's open, aiming to fill these gaps with strategic entry and exit points.

These strategies emphasize preparation, discipline, and risk management to navigate the risks associated with day trading AAPL effectively. Traders should stay informed about market conditions, economic indicators, and company-specific news to make informed decisions and optimize their trading experience.